# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [95]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [96]:
weather_df = pd.read_csv("output_data/cities.csv")
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Zhigansk,66.7697,123.3711,-40.57,79,72,4.73,RU,1610601553
1,Evensk,61.9500,159.2333,-28.21,74,76,2.59,RU,1610601554
2,Juneau,58.3019,-134.4197,3.00,87,75,3.60,US,1610601407
3,Mubende,0.5606,31.3900,20.24,64,79,0.12,UG,1610601558
4,San Ignacio,17.1561,-89.0714,22.00,88,75,1.01,BZ,1610601559


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [99]:
## Creating a heat map that displays the humidity for every city from the part I of the homework

# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Define humidity so can set it as the weight
humidity = weather_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [100]:
# Narrow down the DataFrame to find your ideal weather condition
## Grading rubric states dataframe created contains 10 or less rows, conditions need to narrow down to 10 or less.

In [101]:
# Investigating values in my dataframe, my temperature is in degrees celsius
print(weather_df["Max Temp"].min())
print(weather_df["Max Temp"].max())
print(weather_df["Wind Speed"].min())
print(weather_df["Wind Speed"].max())
print(weather_df["Cloudiness"].min())
print(weather_df["Cloudiness"].max())

-43.94
44.0
0.12
15.36
0
100


In [102]:
# Using 23 to 28 degrees celcius as my ideal temp conditions, to get <10 rows
# Dropping any rows that don't contain all three conditions
ideal_df = weather_df.loc[(weather_df["Max Temp"] >= 23) & (weather_df["Max Temp"] <= 28) & (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] < 1), :]
ideal_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,São João da Barra,-21.6403,-41.0511,24.35,87,0,6.88,BR,1610601607
42,Kanigiri,15.4000,79.5167,26.68,62,0,3.01,IN,1610601627
125,Pakse,15.1167,105.7833,25.00,47,0,2.06,LA,1610601813
129,Bonthe,7.5264,-12.5050,24.81,87,0,1.04,SL,1610601821
195,Cayenne,4.9333,-52.3333,26.00,83,0,2.57,GF,1610601934


In [103]:
ideal_df.count()

City          8
Lat           8
Lng           8
Max Temp      8
Humidity      8
Cloudiness    8
Wind Speed    8
Country       8
Date          8
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [104]:
# Pulling out columns according to grading rubric 
hotel_df = pd.DataFrame(ideal_df, columns=["City", "Country", "Lat", "Lng"])
# Adding an empty column to df 
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
26,São João da Barra,BR,-21.6403,-41.0511,
42,Kanigiri,IN,15.4000,79.5167,
125,Pakse,LA,15.1167,105.7833,
129,Bonthe,SL,7.5264,-12.5050,
195,Cayenne,GF,4.9333,-52.3333,
288,Bubaque,GW,11.2833,-15.8333,
355,Myitkyina,MM,25.3833,97.4000,
479,Boa Vista,BR,2.8197,-60.6733,


In [105]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in ideal_df.iterrows():
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index, "Hotel Name"] = None


In [106]:
# Drop any rows with missing info
hotel_df = hotel_df.dropna(how="any")
hotel_df

,City,Country,Lat,Lng,Hotel Name
26,São João da Barra,BR,-21.6403,-41.0511,OYO Pousada Mediterrâneo
42,Kanigiri,IN,15.4000,79.5167,Hotel
125,Pakse,LA,15.1167,105.7833,Champasak Grand Hotel
129,Bonthe,SL,7.5264,-12.5050,Bonthe Holiday Village
195,Cayenne,GF,4.9333,-52.3333,Royal Amazonia
288,Bubaque,GW,11.2833,-15.8333,BOB FISHING CLUB BIJAGOS
355,Myitkyina,MM,25.3833,97.4000,Palm Spring Resort Hotel Myitkyina
479,Boa Vista,BR,2.8197,-60.6733,Hotel Ajuricaba


In [107]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [108]:
# Add marker layer ontop of heat map, info_box_content makes pins clickable
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))